In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


En este notebook trabajaremos con las librerías por default de tensorflow. Descargaremos a trabés de tensorflow_datasets

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews",with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5JAECP/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5JAECP/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5JAECP/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
imdb

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [4]:
imdb['train']

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

Al parecer, al entrar a imdb['train'] nos bota dos valores: uno para el string y otro para el label.

In [0]:
#Armaremos nuestro train_data y test_data. 
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
testing_sentences = []

training_labels = []
testing_labels = []

for string,label in train_data:
    training_sentences.append(str(string.numpy()))
    training_labels.append(label.numpy())

for string,label in test_data:
    testing_sentences.append(str(string.numpy()))
    testing_labels.append(label.numpy())

training_labels_final = np.array(training_labels) #Mis labels de entrenamiento final
testing_labels_final = np.array(testing_labels) #Mis labels de testeo final

In [6]:
print('Length traninig labels:', len(training_labels), '\n Length testing labels:', len(testing_labels))
print('Length traninig sentences:', len(training_sentences), '\n Length testing labels:', len(testing_sentences))

Length traninig labels: 25000 
 Length testing labels: 25000
Length traninig sentences: 25000 
 Length testing labels: 25000


In [0]:
vocab_size = 10000 #El número de palabras que voy a definir como máximas para mi data
embedding_dim = 16 #La dimensión de vectores que tendré al embeber? 
max_length = 120 # El número de dimensiones que tendrán mi padding
trunc_type = 'post' #Quiero que se pongan los ceros para la derecha
oov_tok = "<OOV>" #Cuando en el test me toque una palabra que no estaba en el train

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [21]:
#Por ejemplo, ahora tengo la data de entrenamiento y testeo padeada
padded.shape

(25000, 120)

In [13]:
testing_padded

array([[  11, 1450, 1528, ...,   60,   48,  214],
       [  26, 5473,    3, ...,  971,    1,    1],
       [  21,    4,  615, ..., 1181, 1023,   47],
       ...,
       [   6,  107,   87, ...,   64,  522,  555],
       [   0,    0,    0, ...,   54,   74,  220],
       [   0,    0,    0, ...,  458, 8360, 2149]], dtype=int32)

In [19]:
#También podemos ver que el word_index está hecho. Pero, ahora tendremos que voltear los valores y llaves. 
reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])

#Esta función nos va a servir para trasladar todo a los términos que tenemos en nuestro word_index, para poder saltarnos las comas y mayúsculas, etc.

#Si en caso no esté el valor (osea que sea 0), entonces pondremos '?' en la codificación
def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
#Los dos son lo mismo, solo que el de arriba está paddeado y el de abajo es el original. Junto con comas y puntuanciones.
print(training_sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

In [0]:
#Construiremos nuestro modelo secuencial que optará por una capa de embedding al princiio, y lo demás una red con un hidden layer de 6 neuronas. 
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 18)           180000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 2160)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 12966     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 192,973
Trainable params: 192,973
Non-trainable params: 0
_________________________________________________________________


In [28]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs = num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 244us/sample - loss: 0.4888 - accuracy: 0.7480 - val_loss: 0.3654 - val_accuracy: 0.8365
Epoch 2/10
25000/25000 [==============================] - 6s 222us/sample - loss: 0.2282 - accuracy: 0.9125 - val_loss: 0.3882 - val_accuracy: 0.8313
Epoch 3/10
25000/25000 [==============================] - 6s 241us/sample - loss: 0.0699 - accuracy: 0.9842 - val_loss: 0.4740 - val_accuracy: 0.8229
Epoch 4/10
25000/25000 [==============================] - 6s 222us/sample - loss: 0.0171 - accuracy: 0.9981 - val_loss: 0.5348 - val_accuracy: 0.8247
Epoch 5/10
25000/25000 [==============================] - 6s 225us/sample - loss: 0.0054 - accuracy: 0.9995 - val_loss: 0.5979 - val_accuracy: 0.8242
Epoch 6/10
25000/25000 [==============================] - 6s 241us/sample - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.6429 - val_accuracy: 0.8251
Epoch 7/10
25000/25000 [==========================

In [29]:
e = model.layers[0]
weights = e.get_weights()[0]
weights.shape # Esto será el (vocab_size, embedding_dim)

(10000, 18)

In [34]:
#Se supone que este es el vector de la palabra 54.
weights[54]

array([-0.09355114, -0.01439267,  0.14008147, -0.03616806, -0.06947594,
       -0.01141866,  0.08891572,  0.00915763,  0.06953935,  0.08576518,
        0.09251753, -0.0874972 ,  0.07871574, -0.10978299, -0.0811759 ,
       -0.15868841, -0.06979665, -0.0624471 ], dtype=float32)

In [35]:
reverse_word_index[54]

'very'

In [0]:
#Ahora vamos a expportaro lo embebido en un archivo tsv

import io

out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')

#Como hay 10mil palabras, iteraré por encima de eso
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num] #Agarra la palabra
  embeddings = weights[word_num] #Agarra el vector que corresponde a esa palabra
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [0]:
try: 
  from google.colab import files
  
except ImportError:
  print('gaaaa')
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')